In [21]:
import warnings
warnings.filterwarnings('ignore') # warningが出ないように設定

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline


pd.set_option("display.max_rows", None) # pandasの表示上限をなくす
pd.set_option("display.max_columns", None) # pandasの表示上限をなくす

## データのローディング

In [23]:
data_path = '../input/'

data = pd.read_csv(data_path + 'data_for_cleansing.csv')

data

### データの説明 

| カラム | 説明 |
| --- | --- |
| *air_store_id* | 店舗固有のID |
| *visit_date* | 来客した日付 |
| *visitors* | 来客数(目的変数) |
| *air_genre_name*   | レストランの種別 |
| *air_area_name* | レストランの場所文字列 |
| *latitude* | レストランの緯度 |
| *longitude* | レストランの緯度 |

##### 資料へ...

## 表記ゆれ処理

### 表記ゆれ確認
- ヒアリングした内容を確認

In [ ]:
# air_store_idの先頭3文字を取得し、データ中にそれらが幾つ含まれているかを表示
data["air_store_id"].apply(lambda x: x[:3]).value_counts().head(10)

In [ ]:
data

##### 資料へ...

### 対策コード
- 先方の要望通り*air*に統一

In [ ]:
# idを条件分岐し、正しいidを返す関数」
def revise_failed_id(x):
    start_word = x[:3]
    if start_word == "air":
        return x
    elif start_word == "AIR":
        return x.replace("AIR", "air")
    elif start_word == "ａｉｒ":
        return x.replace("ａｉｒ", "air")
    elif start_word == "ＡＩＲ":
        return x.replace("ＡＩＲ", "air")
    else:
        return np.nan
data["air_store_id"]=data["air_store_id"].astype(str)
data["air_store_id"] = data["air_store_id"].apply(revise_failed_id)
data["air_store_id"]=data["air_store_id"].astype(str)

In [ ]:
# 再度確認
data["air_store_id"].apply(lambda x: x[:3]).value_counts()

##### 資料へ...

## 欠損値

In [ ]:
data.head()

### 欠損値可視化
- 表として出力
- 図として出力

In [ ]:
## 欠損値の数と割合を返す
def missing_columns(df):
    total = df.isnull().sum().sort_values(ascending = False)
    percent = (df.isnull().sum()/df.isnull().count()*100).sort_values(ascending = False)
    missing_data  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    return missing_data

In [ ]:
missing_columns(data)

In [ ]:
import missingno as msno

msno.matrix(data)

#### 発見
- *visitors*に0.2%の欠損が存在

##### 資料へ...

### 対策コード
- 欠損タイプがMCARのため、今回は簡単に実装ができるリストワイズ法を選択

In [ ]:
data = data.dropna(how='any') # 「how='any'」でリストワイズ法を行うことができる

In [ ]:
msno.matrix(data)

##### 資料へ...

## カテゴリカル

### カテゴリカル変数確認

In [ ]:
data.head()

In [ ]:
data.dtypes

### 対策コード
- *air_genre_name*と*air_area_name*はLabelEncoding
- *visit_date*は「年、月、日、週」に分割しEncoding

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
data["air_genre_name_le"] = le.fit_transform(data["air_genre_name"])
data["air_area_name_le"] = le.fit_transform(data["air_area_name"])

data.head(2)

In [ ]:
# pandas.to_datetime で str型からdatetime型 へ変換
data["visit_date"] = pd.to_datetime(data['visit_date'])
data["visit_year"] = data["visit_date"].dt.year
data["visit_month"] = data["visit_date"].dt.month
data["visit_day"] = data["visit_date"].dt.day
data["visit_weekday"] = data["visit_date"].dt.weekday

data.head(3)

## 学習データ
- カラムをフィルターしてtrain test splitを行う

In [ ]:
# trainとtestを分割し、目的変数と説明変数を更に分割して、dictで返す.(date_colで日付カラム指定、targetで説明変数カラムを指定.)
def generate_train_test(df, drop_list, train_test_split_date,date_col,target):
    train = df[df[date_col] <= train_test_split_date]
    test = df[df[date_col] > train_test_split_date]

    train = train.drop(drop_list, axis=1)
    test = test.drop(drop_list, axis=1)

    X_train = train.ix[:, train.columns != target]
    X_test = test.ix[:, test.columns != target]
    y_train = train[target]
    y_test = test[target]
    
    return {"X_train":X_train, "X_test":X_test, "y_train":y_train, "y_test":y_test}

In [ ]:
from datetime import timedelta

TRAIN_SIZE_RATE = 0.8

during_days = (data["visit_date"].max() - data["visit_date"].min()).days

train_test_split_date = data["visit_date"].min() + timedelta(days=int(during_days * TRAIN_SIZE_RATE))

print("train_test_split_date is \"{0}\"".format(train_test_split_date))

#### カラムのフィルター(削除するカラムの説明)

| カラム | 説明 |
| --- | --- |
| *air_store_id* | 未知のデータに対応できないため |
| *visit_date* | object(str)型のため |
| *air_genre_name* | object(str)型のため |
| *air_area_name*   | object(str)型のため |
| *latitude* | *air_store_id*と紐づく可能性がある |
| *longitude* | 同上 |
| *air_area_name_le* | 同上 |

In [ ]:
# 学習データには使用しない(できない)特徴量を削除
drop_list = ["air_store_id", "visit_date", "air_genre_name", "air_area_name", "latitude", "longitude", "air_area_name_le"]
date_col="visit_date"
target="visitors"
dataset = generate_train_test(data, drop_list, train_test_split_date,date_col,target)

print("X_train")
display(dataset["X_train"].head(3))
print("X_test")
display(dataset["X_test"].head(3))
print("y_train")
display(dataset["y_train"].head(3))
print("y_test")
display(dataset["y_test"].head(3))

## データの保存
- DataFrameはcsvで保存することができるが、今回はpickleで保存
    - [reference](https://docs.python.jp/2/library/pickle.html)

In [ ]:
import pickle

In [ ]:
with open("../pickle/data_step1.pickle", mode="wb") as f:
    pickle.dump(data, f)

In [ ]:
with open("../pickle/dataset_step1.pickle", mode="wb") as f:
    pickle.dump(dataset, f)